In [5]:
import parmed
import logging
import os, sys, copy
import numpy as np

from simtk.openmm.app import *
from simtk.openmm import *
from simtk import unit

In [12]:
struct.save('../sgill_testsystem/ethlyene_structure.pdb')

In [6]:
# Create our system
struct1 = parmed.load_file('../sgill_testsystem/sqB.pdb')
struct2 = parmed.load_file('../sgill_testsystem/eth.prmtop', 
                           xyz='../sgill_testsystem/eth.inpcrd')
struct = struct1 + struct2
struct.save('../sgill_testsystem/ethlyene_structure.pdb')
system = struct.createSystem(nonbondedMethod=NoCutoff, constraints=HBonds, 
                             removeCMMotion=True)

# Remove the nonbonded forces on the charged particles
system.removeForce(4)
nonbonded = system.getForce(3)
numParticles = system.getNumParticles()
rangeparticles = range(numParticles)
parameter_list = [nonbonded.getParticleParameters(i) for i in rangeparticles]
system.removeForce(3)

In [7]:
# Create a Custom Force for alchemical parameters
pairwiseForce = CustomNonbondedForce("q/(r^2) + 4*epsilon*((sigma/r)^12-(sigma/r)^6); sigma=0.5*(sigma1+sigma2)*lambda_sterics; epsilon=sqrt(epsilon1*epsilon2)*lambda_electrostatics; q = lambda_charge*(q1*q2)")
pairwiseForce.addPerParticleParameter("sigma")
pairwiseForce.addPerParticleParameter("epsilon")
pairwiseForce.addPerParticleParameter("q")
pairwiseForce.addPerParticleParameter("lambda_on")
pairwiseForce.addGlobalParameter("lambda_sterics", 1)
pairwiseForce.addGlobalParameter("lambda_electrostatics", 1)
pairwiseForce.addGlobalParameter("lambda_charge", 1)

2

In [8]:
# Define nonbonded forces between two particles and ethylene
for i, parameter in enumerate(parameter_list):
    new_param = [parameter[1]*1.2] + [parameter[2]]+[parameter[0]]
    pairwiseForce.addParticle()
    pairwiseForce.setParticleParameters(i,new_param+[0])
pairwiseForce.setParticleParameters(0, [0.324999852378,0.71128, -0.2, 10])
pairwiseForce.setParticleParameters(1, [0.324999852378,0.71128, -0.5, 10])

pairwiseForce.addInteractionGroup([0,1], rangeparticles[2:])
num_params = pairwiseForce.getNumPerParticleParameters()
system.addForce(pairwiseForce)

3

In [9]:
# Add a restraining force to keep ethylene in the center
centoridForce = CustomCentroidBondForce(2, ('0.5*k*distance(g1,g2)^2'))
centoridForce.addPerBondParameter('k')
centoridForce.addGroup([0,1], [1,1])
centoridForce.addGroup([2,3])
bondGroups = [0,1]
bondParameters=[100000]
centoridForce.addBond(bondGroups, bondParameters)
system.addForce(centoridForce)
system.setParticleMass(0, 0)
system.setParticleMass(1, 0)

In [11]:
# Serialize and save our system
serialized_system = openmm.XmlSerializer.serialize(system)
with open('../sgill_testsystem/ethylene_system.xml', 'w') as outfile:
    outfile.write(serialized_system)

In [ ]:
struc